# LIN 313 Final Project
_**By Jed Wang**_

In [ ]:
# if True, complete tokenizer; if False, verified tokenizer
complete = False

**_FINALLY!_ I get to fine-tune!**

In [ ]:
!pip install datasets tensorflow evaluate
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.9 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

## A. Building the tokenizer
Just like all the other times

In [ ]:
!wget -O tokenizer_full.json https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json

--2023-11-28 05:57:39--  https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json
Resolving gist.github.com (gist.github.com)... 140.82.121.4
Connecting to gist.github.com (gist.github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json [following]
--2023-11-28 05:57:39--  https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6471221 (

In [ ]:
!wget -O tokenizer_veri.json https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json

--2023-11-28 05:57:40--  https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json
Resolving gist.github.com (gist.github.com)... 140.82.121.4
Connecting to gist.github.com (gist.github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json [following]
--2023-11-28 05:57:40--  https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6196696 (

In [ ]:
from transformers import PreTrainedTokenizerFast

# Instantiate the tokenizer from before
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer_full.json" if complete else "tokenizer_veri.json",
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
    model_max_length=512
)

## B. Import the data
This is the pretokenized stuff from before.

In [ ]:
# the dataset to fetch
dataset_name = "AFQMC"

In [ ]:
from datasets import load_dataset
import multiprocessing

dataset = load_dataset(f"jedwang/{dataset_name}",
                       num_proc=multiprocessing.cpu_count())
dataset = dataset.rename_column("score", "label")
# { e[0]: len(e[1]) for e in dataset.unique("label").items() }, len(dataset["train"])
dataset

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Setting num_proc from 8 back to 1 for the test split to disable multiprocessing as it only contains one shard.


Generating test split:   0%|          | 0/3861 [00:00<?, ? examples/s]

Setting num_proc from 8 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split:   0%|          | 0/34334 [00:00<?, ? examples/s]

Setting num_proc from 8 back to 1 for the validation split to disable multiprocessing as it only contains one shard.


Generating validation split:   0%|          | 0/4316 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3861
    })
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 34334
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4316
    })
})

In [ ]:
num_labels = 2
id2label = { i: label for i, label in enumerate(dataset["train"].features["label"].names) }
label2id = { label: i for i, label in enumerate(dataset["train"].features["label"].names) }
id2label, label2id

({0: '0', 1: '1'}, {'0': 0, '1': 1})

## C. Import the model
This is the pretrained stuff from before.

In [ ]:
from transformers import AutoModelForSequenceClassification

# set to True if continuing fine-tuning
continue_training = False

if not continue_training or model not in globals():
  model = AutoModelForSequenceClassification.from_pretrained(
      "jedwang/bert-base-split-chinese",
      revision=dataset_name if continue_training else "quart_x3",

      num_labels=num_labels,
      id2label=id2label,
      label2id=label2id
  )

config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jedwang/bert-base-split-chinese and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## D. Other training schenanigans
Other stuff I have to set up (eval, data_collator, etc.).

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## E. Perform the training!
Wooooooooooooooooo
> Shoutouts to [#107960](https://github.com/pytorch/pytorch/issues/107960) for the fix again

In [ ]:
# https://github.com/pytorch/pytorch/issues/107960
# !ldconfig -p | grep libcuda
# !find /usr -name 'libcuda.so'
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link



In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="pretrain",
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id=f"jedwang/{dataset_name}-checkpoints",
    hub_private_repo=True,

    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
    weight_decay=0.01,

    gradient_accumulation_steps=16,
    per_device_train_batch_size=16,
    num_train_epochs=5,

    evaluation_strategy="epoch",
    save_strategy="epoch",

    learning_rate=2e-3,
    warmup_ratio=0.02,
    # max_steps=562,
    fp16=True, # enable on GPUs
    # torch_compile=True,
    # dataloader_num_workers=2,

    disable_tqdm=False
)

In [ ]:
if True:
  !export CUDA_LAUNCH_BLOCKING=1

In [ ]:
if False:
  !pip install torch --extra-index-url https://download.pytorch.org/whl/nightly/cu116/

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,

    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],

    # tokenizer=tokenizer,
    data_collator=data_collator,

    compute_metrics=compute_metrics,
)

RuntimeError: ignored

Magic sauce begin!

In [ ]:
trainer.train()

RuntimeError: ignored

Make sure we save

In [ ]:
trainer.push_to_hub(
    commit_message=f"Completed fine-tuning on {dataset_name}"
)
model.push_to_hub(
    repo_id="jedwang/bert-base-split-chinese",
    revision=dataset_name,
    commit_description=f"Completed fine-tuning for {dataset_name}!"
)